Using a Convoluted network to predict dog-breed classification. Try three steps:

- Build CNN
- Add image pre-processing
- Use pre-trained model

In [1]:
import pandas as pd
import os, shutil
import numpy as np

In [2]:
import tensorflow
from tensorflow.compat.v1.keras.backend import set_session
config = tensorflow.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tensorflow.compat.v1.Session(config=config)
set_session(sess)

In [3]:
os.chdir("C:/Users/abbal/Documents/Py - Dog Breed Identification")

In [ ]:
## get file with labels
File_Labels = pd.read_csv("labels.csv")

Unique_Breeds = File_Labels["breed"].unique()
print(Unique_Breeds)

In [ ]:
pd.crosstab(File_Labels["breed"], "count")

In [ ]:
## Create a folder for each dog breed
TTV_Sets = ["test", "train", "validation"]
for st in TTV_Sets:
    os.mkdir(os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized" , st))

for st in TTV_Sets:
    for i in Unique_Breeds:     
        os.mkdir(os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized",st, i))

In [ ]:
## split labels to train, test and validation
def train_validate_test_split(df, train_percent=.6, validate_percent=.2, seed=None):
    np.random.seed(seed)
    perm = np.random.permutation(df.index)
    m = len(df.index)
    train_end = int(train_percent * m)
    validate_end = int(validate_percent * m) + train_end
    train = df.iloc[perm[:train_end]]
    validate = df.iloc[perm[train_end:validate_end]]
    test = df.iloc[perm[validate_end:]]
    return train, validate, test

In [ ]:
train, validate, test = train_validate_test_split(File_Labels)

In [ ]:
## Move all files to their respective folder
for fl in range(train.shape[0]):
    Curr_FileName = train.iloc[fl, 0]
    Curr_FileName = '{}.jpg'.format(Curr_FileName)
    Curr_DogBreed = train.iloc[fl, 1]
    Orig_file = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/train", Curr_FileName)
    New_file = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized/train",Curr_DogBreed, Curr_FileName)
    shutil.copyfile(Orig_file, New_file)

for fl in range(test.shape[0]):
    Curr_FileName = test.iloc[fl, 0]
    Curr_FileName = '{}.jpg'.format(Curr_FileName)
    Curr_DogBreed = test.iloc[fl, 1]
    Orig_file = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/train", Curr_FileName)
    New_file = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized/test",Curr_DogBreed, Curr_FileName)
    shutil.copyfile(Orig_file, New_file)
    
for fl in range(validate.shape[0]):
    Curr_FileName = validate.iloc[fl, 0]
    Curr_FileName = '{}.jpg'.format(Curr_FileName)
    Curr_DogBreed = validate.iloc[fl, 1]
    Orig_file = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/train", Curr_FileName)
    New_file = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized/validation",Curr_DogBreed, Curr_FileName)
    shutil.copyfile(Orig_file, New_file)

In [4]:
train_dir = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized", 'train')
validation_dir = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized", 'validation')
test_dir = os.path.join("C:/Users/abbal/Documents/Py - Dog Breed Identification/Categorized", 'test')

In [9]:
from tensorflow.keras import layers
from tensorflow.keras import models

Create CNN and output summary

In [10]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 148, 148, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 74, 74, 32)        0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 72, 72, 64)        18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 36, 36, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 34, 34, 128)       73856     
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 17, 17, 128)       0         
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 15, 15, 128)       1

In [11]:
# because an image can only obtain one classification, use sparse categorical
from tensorflow.keras import optimizers

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [12]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

# All images will be rescaled by 1./255
train_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=20,
        class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=20,
        class_mode='sparse')

Found 6133 images belonging to 120 classes.
Found 2044 images belonging to 120 classes.


In [13]:
for data_batch, labels_batch in train_generator:
    print('data batch shape:', data_batch.shape)
    print('labels batch shape:', labels_batch.shape)
    break

data batch shape: (20, 150, 150, 3)
labels batch shape: (20,)


In [8]:
history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

NameError: name 'model' is not defined

In [14]:
# Poor performance, so let's do some image pre-processing

datagen = ImageDataGenerator(
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

In [19]:
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
                        input_shape=(150, 150, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dropout(0.5))
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='softmax'))

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=1e-4),
              metrics=['acc'])

In [20]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=45,
    width_shift_range=0.2,
    height_shift_range=0.2,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True)

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(150, 150),
        batch_size=32,
        class_mode='sparse')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(150, 150),
        batch_size=32,
        class_mode='sparse')

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50)

Found 6133 images belonging to 120 classes.
Found 2044 images belonging to 120 classes.
Epoch 1/100
100/100 [==============================] - 18s 184ms/step - loss: nan - acc: 0.0113 - val_loss: nan - val_acc: 0.0144
Epoch 2/100
100/100 [==============================] - 17s 167ms/step - loss: nan - acc: 0.0103 - val_loss: nan - val_acc: 0.0144
Epoch 3/100
100/100 [==============================] - 17s 166ms/step - loss: nan - acc: 0.0082 - val_loss: nan - val_acc: 0.0144
Epoch 4/100
100/100 [==============================] - 17s 169ms/step - loss: nan - acc: 0.0147 - val_loss: nan - val_acc: 0.0144
Epoch 5/100
100/100 [==============================] - 17s 167ms/step - loss: nan - acc: 0.0100 - val_loss: nan - val_acc: 0.0144
Epoch 6/100
100/100 [==============================] - 17s 167ms/step - loss: nan - acc: 0.0103 - val_loss: nan - val_acc: 0.0144
Epoch 7/100
100/100 [==============================] - 17s 167ms/step - loss: nan - acc: 0.0113 - val_loss: nan - val_acc: 0.0144
Ep

KeyboardInterrupt: 

In [ ]:
import matplotlib.pyplot as plt

acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']

epochs = range(len(acc))

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()

plt.figure()

plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()

plt.show()

## Use pre-trained network

In [ ]:
from tensorflow.keras.applications import VGG16

conv_base = VGG16(weights='imagenet',
                  include_top=False,
                  input_shape=(250, 250, 3))

In [ ]:
conv_base.summary()

In [ ]:
import os
import numpy as np
from tensorflow.keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(rescale=1./255)
batch_size = 20

def extract_features(directory, sample_count):
    features = np.zeros(shape=(sample_count, 7, 7, 512))
    labels = np.zeros(shape=(sample_count))
    generator = datagen.flow_from_directory(
        directory,
        target_size=(250, 250),
        batch_size=batch_size,
        class_mode='binary')
    i = 0
    for inputs_batch, labels_batch in generator:
        features_batch = conv_base.predict(inputs_batch)
        features[i * batch_size : (i + 1) * batch_size] = features_batch
        labels[i * batch_size : (i + 1) * batch_size] = labels_batch
        i += 1
        if i * batch_size >= sample_count:
            # Note that since generators yield data indefinitely in a loop,
            # we must `break` after every image has been seen once.
            break
    return features, labels

train_features, train_labels = extract_features(train_dir, 2000)
validation_features, validation_labels = extract_features(validation_dir, 1000)
test_features, test_labels = extract_features(test_dir, 1000)

In [ ]:
train_features = np.reshape(train_features, (2000, 7 * 7 * 512))
validation_features = np.reshape(validation_features, (1000, 7 * 7 * 512))
test_features = np.reshape(test_features, (1000, 7 * 7 * 512))

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers
from tensorflow.keras import optimizers

model = models.Sequential()
model.add(layers.Dense(256, activation='relu', input_dim=7 * 7 * 512))
model.add(layers.Dropout(0.5))
model.add(layers.Dense(120, activation='softmax'))

model.compile(optimizer=optimizers.RMSprop(lr=2e-5),
              loss='sparse_categorical_crossentropy',
              metrics=['acc'])

In [ ]:
history = model.fit(train_features, train_labels,
                    epochs=30,
                    batch_size=100,
                    validation_data=(validation_features, validation_labels))

In [ ]:
from tensorflow.keras import models
from tensorflow.keras import layers

model = models.Sequential()
model.add(conv_base)
model.add(layers.Flatten())
model.add(layers.Dense(256, activation='relu'))
model.add(layers.Dense(120, activation='softmax'))

In [ ]:
conv_base.trainable = False

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
      rescale=1./255,
      rotation_range=40,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True,
      fill_mode='nearest')

# Note that the validation data should not be augmented!
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        # This is the target directory
        train_dir,
        # All images will be resized to 150x150
        target_size=(250, 250),
        batch_size=20,
        # Since we use binary_crossentropy loss, we need binary labels
        class_mode='binary')

validation_generator = test_datagen.flow_from_directory(
        validation_dir,
        target_size=(250, 250),
        batch_size=20,
        class_mode='binary')

model.compile(loss='sparse_categorical_crossentropy',
              optimizer=optimizers.RMSprop(lr=2e-5),
              metrics=['acc'])

history = model.fit_generator(
      train_generator,
      steps_per_epoch=100,
      epochs=100,
      validation_data=validation_generator,
      validation_steps=50,
      verbose=2)